## 1.code to implement decision tree.

In [2]:
import math as m
import pandas as pd
import numpy as np

# Creating a decision tree node, which stores the input:-(x), output:-(y), and the index, which is helpful while creating the graph for the decision tree
class DecisionTreeNode:
    def __init__(self, x, y):
        self.x = x            # feature to split upon 0 indexing.
        self.y = y            # output data from node.
        self.children = {}    # This hashmap is basically to store the children of a parent node
        self.index = -1       # Index, which is helpful while creating graph using pydotplus library,it will be used to assign a unique index to each node.
        
    # Function/method to add a child to a node
    def add_child(self, feature, data):  
        self.children[feature] = data   # A new child is added to a node, based on the feature on which it was split upon
        
        
# The main class, with all required functions
class DecisionTree:
    
    # Constructor which initialises a decision tree classfier with a root which initially is None
    def __init__(self):
        self.root = None
        
        
    # Returns a dictionary with keys as unique values of output data and the corresponding value as its frequency of that output data
    def __numUniqueOp(self, y):
        d = {}
        for i in y:
            if i not in d:
                d[i] = 1
            else:
                d[i] += 1
                
        return d
    

    # Function to calculate the gini index of a particular node 
    def __gini_index(self,y):
        possibleOps = self.__numUniqueOp(y)
        gini_ind = 1
        tot = len(y)
        for i in possibleOps:
            p = possibleOps[i]/tot
            gini_ind -= p**2
                                         
        return gini_ind
    
    # Function to calculate the gini gain for a selected feature(f) to split upon
    def __gain_gini(self, x, y, f):
        
        gini_intial = self.__gini_index(y)  # initial gini index before splitting
        gini_final = 0
        possible_x = set(x[:, f]) # All the unique values for the gini index
        df = pd.DataFrame(x)   # Converting the required input into a data frame
        df[df.shape[1]] = y    # Appending the output, for the considered input
        
        initial_size = df.shape[0]
        
        for i in possible_x:
            df1 = df[df[f] == i]   # Selecting the new set of input for the selected feature f and its value i
            size = df1.shape[0]
            
            gini_final += (size/initial_size)*(self.__gini_index(df1[df1.shape[1]-1]))
        
        gini_gain = gini_intial - gini_final
        return gini_gain
    
    
     # Function for calculating the entropy for a given state/node in a decision tree
    def __entropy(self, y):
        possibleOps = self.__numUniqueOp(y)  # All the unique output data possible for given input data
        entro = 0
        tot = len(y)
        for i in possibleOps:  
            pi = possibleOps[i]/tot
            entro += (-pi)*(m.log(pi,2))    
            
        return entro
    
    # Function to calculate the gain ratio for a given input, output and selected feature
    def __gain_ratio(self, x, y, f):
        curr_entro = self.__entropy(y)   # Current entropy of the parent node
        info_new = 0
        split_info = 0
        possible_x = set(x[:, f])        # Required input for selected featire "f"
        df = pd.DataFrame(x)             # Converting the required input into a data frame
        df[df.shape[1]] = y              # Appending the output, for the considered input
        
        initial_size = df.shape[0]      
        
        for i in possible_x:
            df1 = df[df[f] == i]     # Selecting the new set of input for the selected feature f and its value i
            size = df1.shape[0]
            
            info_new += (size/initial_size)*self.__entropy(df1[df1.shape[1]-1])
            split_info += (-size/initial_size)*m.log((size/initial_size), 2)
            
        if split_info == 0:      # To handle 0 denominator error, as Gain ratio = info_gain/split_info
            return math.inf
        
        info_gain = curr_entro - info_new
        return info_gain/split_info
    
    # A helper function, to construct a decision tree
    # This function returns the root of decision tree comstructed from the training data
    def __build_dt_helper(self, x, y, unused_features, possible_ops, lvl, metric):
        # possible_ops are basically the output classes
        # metric parameter has a default value of gain_ratio, but can also be changed to gini_index manually.
        # depth here is level
        
        ### Base cases ###
        
        # If only one output is possible at a node, that means that particular node is pure(leaf node)
        if len(set(y)) == 1:   
            print("Level",lvl)
            op = None
            for i in possible_ops:
                if i in y:
                    op = i
                    print("Count of", i,"=",len(y))
                    
                else:
                    print("Count of", i,"=",0)
                    
            if metric == "gain_ratio":
                print("Current entropy is 0.0")     # For a pure node, entropy = 0
            else:
                print("Current Gini index is 0.0")
            print("Reached leaf Node")
            print()
            return DecisionTreeNode(None,op)    # Returning the node to build the tree
        
        
        # If all the features availble are used ,then we take the majority vote and then classify yhe values in the node(leaf node)
        if len(unused_features) == 0:
            print("Level",lvl)
            possible_vals = self.__numUniqueOp(y)
            op = None
            maj = -m.inf
            for i in possible_ops:
                if i not in possible_vals:
                    print("Count of", i,"=",0)
                    
                else:
                    if possible_vals[i] > maj:
                        maj = possible_vals[i]
                        op = i
                    print("Count of", i,"=",possible_vals[i])
    
            print("Reached leaf Node")
            print()
            return DecisionTreeNode(None,op)    # Returning the node to build a decision tree
        
        
        # To find maximum gain ratio by splitting the with different features and then selecting the best feature
        max_gain = -m.inf
        best_feaure = ""
        for f in unused_features:
            if metric == "gain_ratio":
                curr_gain = self.__gain_ratio(x,y,f)
                
            else:
                curr_gain = self.__gain_gini(x,y,f)
            
            if curr_gain > max_gain:   # if current gain is better than max gain by selecting a feature "f", that feature will be the best feature to split the data upon  
                max_gain = curr_gain
                best_feaure = f
        
        print("Level",lvl)
        possible_vals = self.__numUniqueOp(y)
        op = None
        max_count = -m.inf
        
        for i in possible_ops:
            if i not in possible_vals:
                print("Count of",i,"=",0)
                
            else:
                if possible_vals[i] > max_count:
                    op = i
                    max_count = possible_vals[i]
                    print("Count of",i,"=",possible_vals[i])
                    
    
        if metric == "gain_ratio":
            print("Current entropy is", self.__entropy(y))
            print("Splitting on the feature ",best_feaure," with gain ratio ",max_gain)
        else:
            print("Current Gini index is",self.__gini_index(y))
            print("Splitting on the feature ",best_feaure," with gain gain ",max_gain)
        
        print()
                                         
                                         
        # unique_x here represents the unique values of the input for the feature selected
        unique_x = set(x[:, best_feaure])
        df = pd.DataFrame(x)  
        df[df.shape[1]] = y
                                        
        # Adding a new node by selecting the best feature and passing the output 
        curr_node = DecisionTreeNode(best_feaure, op)
        
        # Removing the selected feature from the unsued features 
        unused_features.remove(best_feaure)
        
        
        for i in unique_x:
            df1 = df[df[best_feaure] == i]
            x_new = df1.iloc[:,0:df1.shape[1]-1].values  
            y_new = df1.iloc[:,df1.shape[1]-1].values
            
            # Performing a pre-order traversal to continue the process
            child_node = self.__build_dt_helper(x_new,y_new, unused_features,possible_ops ,lvl+1, metric)  
            
            # Adding the child node to the current new node
            curr_node.add_child(i, child_node)
            
        # Returning the current node, basically the head of the decision tree
        return curr_node
    
    
    # Fuction to fit the training data, metric here is default set to entropy
    def fit(self, x, y, metric = "entropy"):
        unused_features = [f for f in range(len(x[0]))]
        possible_ops = set(y)
        
        if metric == "entropy":
            metric = "gain_ratio"
            
        else:
            metric = "gini_index"
        self.__root = self.__build_dt_helper(x, y, unused_features, possible_ops , 0, metric)
        
        
    # Helper function to predict data
    def __predict_Helper(self, data, node):
        # Base case
        if len(node.children) == 0:
            return node.y
        
        # selecting the data on which the feature is split upon
        curr_val = data[node.x]
           
        # Returning the output
        if curr_val not in node.children:
            return node.y
        
        # recursively predicting for other given data as well 
        return self.__predict_Helper(data, node.children[curr_val])
        
                                         
    # Predicts for the given data
    def predict(self, x):
        y = np.array([0]*len(x))
        for i in range(len(x)):
            y[i] = self.__predict_Helper(x[i], self.__root)

        return y
    
    # Fucntion to return the score of the classifer
    def score(self, x_test, y_true):
        y_pred = self.predict(x_test)
        count = 0
        for i in range(len(y_pred)):
            if y_pred[i] == y_test[i]:
                count += 1
                
        return count/len(y_pred)
    
    # Fuction for creating a visual representation of Decision Tree using pydotplus
    def export_tree(self, filename = None):
        import pydotplus
        import queue
        
        # Level order traversal on the entire tree and then printing them into a graphiz graph
        count = 0
        dot_data = '''digraph Tree {node [shape=box] ;'''
        q = queue.Queue()
    
        root = self.__root
        q.put(root)
        if root.index == -1:
            root.index = count
            
        dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(count,root.x,root.y)
        
        while (not q.empty()):
            curr_node = q.get()
            for i in curr_node.children:
                count += 1
                if (curr_node.children[i].index == -1):
                    curr_node.children[i].index = count
                    
                # Connecting the children to the node(parent)
                dot_data += "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(curr_node.children[i].index,curr_node.children[i].x,curr_node.children[i].y)
                
                dot_data += "\n{} -> {} [ headlabel=\"Feature value = {}\"]; ".format(curr_node.index,curr_node.children[i].index,i)
                
                # Adding the children nodes into the queue
                q.put(curr_node.children[i])
                
        dot_data += "\n}"
        
                                         
        if filename != None:    
            graph = pydotplus.graph_from_dot_data(dot_data)
            graph.write_pdf(filename)    
        
        # Returing the graph in form of a dotdata format
        return dot_data

In [6]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Splitting the trianing and testing data using a seed
iris = datasets.load_iris()
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)

# Creating an object of the decison tree classifier built above
dt = DecisionTree()

# Fitting the input data and printing as specified in the instructions
dt.fit(x_train,y_train)

Level 0
Count of 0 = 37
Count of 2 = 41
Current entropy is 1.5807197138422104
Splitting on the feature  3  with gain ratio  0.35918191528212107

Level 1
Count of 0 = 23
Count of 1 = 0
Count of 2 = 0
Current entropy is 0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 4
Current entropy is 0.9182958340544896
Splitting on the feature  2  with gain ratio  0.40783617806826955

Level 2
Count of 0 = 0
Count of 1 = 1
Count of 2 = 0
Current entropy is 0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 0
Count of 2 = 1
Current entropy is 0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 1
Count of 2 = 0
Current entropy is 0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 0
Count of 2 = 1
Current entropy is 0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 2
Count of 2 = 0
Current entropy is 0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 0
Count of 2 = 6
Current entropy is 0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 0
Count of

## 2.representation of iris data set.

In [7]:
## create a graphical representation from the generated result in form of an object.
print(dt.export_tree(filename = "irisrep.pdf"))

digraph Tree {node [shape=box] ;
0 [label="Feature to split upon : X[3]\nOutput at this node : 2" ];
1 [label="Feature to split upon : X[None]\nOutput at this node : 0" ];
0 -> 1 [ headlabel="Feature value = 0.2"]; 
2 [label="Feature to split upon : X[2]\nOutput at this node : 1" ];
0 -> 2 [ headlabel="Feature value = 1.5"]; 
3 [label="Feature to split upon : X[None]\nOutput at this node : 2" ];
0 -> 3 [ headlabel="Feature value = 2.1"]; 
4 [label="Feature to split upon : X[None]\nOutput at this node : 2" ];
0 -> 4 [ headlabel="Feature value = 2.3"]; 
5 [label="Feature to split upon : X[None]\nOutput at this node : 1" ];
0 -> 5 [ headlabel="Feature value = 1.2"]; 
6 [label="Feature to split upon : X[None]\nOutput at this node : 0" ];
0 -> 6 [ headlabel="Feature value = 0.6"]; 
7 [label="Feature to split upon : X[None]\nOutput at this node : 1" ];
0 -> 7 [ headlabel="Feature value = 1.0"]; 
8 [label="Feature to split upon : X[1]\nOutput at this node : 2" ];
0 -> 8 [ headlabel="Feature v

In [8]:
dt.score(x_test,y_test)

0.9736842105263158

## create a decision tree for or gate. (output is named as orgate.pdf) will be in documents.

In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

data=np.array([[1,1,1],
             [0,0,0],
             [1,0,1],
             [0,1,1]])
x=data[:,:-1]
y=data[:,-1].astype(str)
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=1)
clf=DecisionTreeClassifier(criterion="entropy")
clf.fit(x_train,y_train)
a=np.array(['0','1'])

DecisionTreeClassifier(criterion='entropy')

In [29]:
from sklearn.tree import export_graphviz
import pydotplus
d=export_graphviz(clf,out_file=None,rounded=True,filled=True,class_names=a)
g=pydotplus.graph_from_dot_data(d)
g.write_pdf("orgate.pdf")

True

## create a decision tree using pydotplus and graphviz for iris dataset 
## output will be in irisset.pdf

In [23]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Splitting the trianing and testing data using a seed
iris = datasets.load_iris()
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)
clf=DecisionTreeClassifier(criterion="entropy")
clf.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy')

In [24]:
from sklearn.tree import export_graphviz
import pydotplus
d=export_graphviz(clf,out_file=None,rounded=True,filled=True,feature_names=iris.feature_names,class_names=iris.target_names)
g=pydotplus.graph_from_dot_data(d)
g.write_pdf("orgate.pdf")

True

In [27]:
type(iris.target_names[0])

numpy.str_